In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from deep_translator import GoogleTranslator
import ast

d:\codingproject\Fastwork - Freelance Projek Rekomendation\env\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [11]:
df_informatika = pd.read_csv("data/Freelance_Teknik_Informatika_Data.csv")
df_bisnis = pd.read_csv("data/Freelance_Manajemen_Bisnis_Data.csv")
df_desain = pd.read_csv("data/Freelance_Desain_Grafis_Data.csv")

In [12]:
df_informatika

,title,price,description,link
0,Android Poker & Slots Game,€250-750 EUR,I Am Looking For A Game Developer That Has Exp...,https://www.freelancer.co.id/projects/android/...
1,Android Gaming App Play Store Upload,₹1500-12500 INR,I need an experienced developer to successfull...,https://www.freelancer.co.id/projects/android/...
2,High-Quality Social Networking App Developer,$750-1500 CAD,I'm looking for a skilled app developer who ha...,https://www.freelancer.co.id/projects/iphone-a...
3,Predictive Analytics from Tabular Data,₹600-1500 INR,I need a skilled machine learning specialist t...,https://www.freelancer.co.id/projects/google-a...
4,Advanced E-commerce Joomla Developer,₹3000-5000 INR,I'm seeking a skilled Joomla developer to work...,https://www.freelancer.co.id/projects/joomla/a...
...,...,...,...,...
309,React Native Developer for Android App,₹600-1500 INR,I'm in need of a highly skilled React Native d...,https://www.freelancer.co.id/projects/android/...
310,Urgent Android App Development Needed,$15-25 USD / hour,I need an Android mobile application developed...,https://www.freelancer.co.id/projects/android/...
311,React Native Developer for Cross-Platform App,$30-250 SGD,I'm in need of a skilled React Native develope...,https://www.freelancer.co.id/projects/iphone-a...
312,Multiplayer 2D Ludo Game for Android,₹600-1500 INR,I'm looking for an experienced game developer ...,https://www.freelancer.co.id/projects/android/...


In [96]:
df_informatika['metadata'] = df_informatika['title'] + ' and the description: ' + df_informatika['description'] + "| Price: " + df_informatika["price"] + "| link: " + df_informatika["link"]
df_bisnis['metadata'] = df_bisnis['title'] + ' and the description: ' + df_bisnis['description'] + "| Price: " + df_bisnis["price"] + "| link: " + df_bisnis["link"]
df_desain['metadata'] = df_desain['title'] + ' and the description: ' + df_desain['description'] + "| Price: " + df_desain["price"] + "| link: " + df_desain["link"]

In [97]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

df_informatika = model.encode(df_informatika["metadata"])
df_bisnis = model.encode(df_bisnis["metadata"])
df_desain = model.encode(df_desain["metadata"])

D:\codingproject\Fastwork - Freelance Projek Rekomendation\env\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [99]:
import json

with open("data_meta/informatika_embed.json", "w") as f:
    data = json.dump(df_informatika.tolist(), f)

with open("data_meta/bisnis_embed.json", "w") as f:
    data = json.dump(df_bisnis.tolist(), f)

with open("data_meta/desain_embed.json", "w") as f:
    data = json.dump(df_desain.tolist(), f)

In [101]:
similarity_vector_data = {
    "Teknik Informatika": "data_meta/informatika_embed.json",
    "Manajemen Bisnis": "data_meta/bisnis_embed.json",
    "Desain Grafis": "data_meta/desain_embed.json"
}

def recommendation_freelance(predict, user_input):
    # Load the corresponding data
    with open(similarity_vector_data[predict], "r") as f:
        data = json.load(f)
    
    # Ensure data is a 2D array
    data = np.array(data)
    
    # Translate and encode the user input
    user_input_translated = GoogleTranslator(target='en').translate(user_input)
    user_embedding = model.encode([user_input_translated])
    
    # Ensure the user embedding is 2D
    user_embedding = np.array(user_embedding).reshape(1, -1)
    
    # Calculate cosine similarity
    similarities = cosine_similarity(user_embedding, data)
    
    # Find the index of the most similar project
    top_3_indices = np.argsort(similarities[0])[-3:][::-1]

    df = pd.read_csv("data/Freelance_Manajemen_Bisnis_Data.csv")
    similarity_data = df.loc[top_3_indices]
    
    return similarity_data

In [103]:
# Example usage
most_similar_index = recommendation_freelance(
    "Manajemen Bisnis", 
    "Saya memiliki kemampuan dalam melakukan pemukuan dalam ranah perusahaan, saya pernah bekerja di bidang Finance untuk menghandle keuangan di perusahaan tersebut."
)

most_similar_index

,title,price,description,link
235,Financial PR Specialist Needed,₹1500-12500 INR,"I am seeking a Public Relations professional, ...",https://www.freelancer.co.id/projects/financia...
100,IT Sales Specialist for Software Projects,₹1500-12500 INR,I'm looking for a driven sales professional wh...,https://www.freelancer.co.id/projects/lead-gen...
76,Cold Caller for Accounting Services Startup,₹600-1500 INR,I'm seeking a skilled cold caller to generate ...,https://www.freelancer.co.id/projects/telemark...


In [4]:
df_bisnis = pd.read_csv("data/Freelance_Manajemen_Bisnis_Data.csv")
df = df_bisnis.loc[[235, 212]]

In [8]:
df

,title,price,description,link
235,Financial PR Specialist Needed,₹1500-12500 INR,"I am seeking a Public Relations professional, ...",https://www.freelancer.co.id/projects/financia...
212,Proactive Sales Rep Wanted for Moving Co.,$10000-20000 USD,"I'm seeking a self-motivated, target-driven Sa...",https://www.freelancer.co.id/projects/market-r...


In [10]:
print(df.iloc[0]["title"])
print(df.iloc[0]["price"])
print(df.iloc[0]["description"])
print(df.iloc[0]["link"])

Financial PR Specialist Needed
₹1500-12500 INR
I am seeking a Public Relations professional, ideally with significant experience in the finance industry. The successful applicant will be responsible for developing and executing press releases and managing media relations, catering specifically to industry professionals. 

Key Tasks:
- Press release creation and distribution
- Building and maintaining media relationships within the financial industry

Ideal Skills:
- Experience in finance-related PR
- Proficiency in tailoring messages for industry professionals
- Strong communication and networking abilities

If you have a proven track record of creating palpable buzz in the finance industry and are adept at targeting industry professionals, I am looking forward to your bid.
https://www.freelancer.co.id/projects/financial-research/financial-specialist-needed
